In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.4347217082977295, Final Batch Loss: 2.2140936851501465
Epoch 2, Loss: 4.434186220169067, Final Batch Loss: 2.2177114486694336
Epoch 3, Loss: 4.431804656982422, Final Batch Loss: 2.2137868404388428
Epoch 4, Loss: 4.428043603897095, Final Batch Loss: 2.2165467739105225
Epoch 5, Loss: 4.425544500350952, Final Batch Loss: 2.204549551010132
Epoch 6, Loss: 4.42397403717041, Final Batch Loss: 2.2126262187957764
Epoch 7, Loss: 4.415851354598999, Final Batch Loss: 2.201073169708252
Epoch 8, Loss: 4.414262294769287, Final Batch Loss: 2.20589280128479
Epoch 9, Loss: 4.414660215377808, Final Batch Loss: 2.205204963684082
Epoch 10, Loss: 4.407128572463989, Final Batch Loss: 2.202409505844116
Epoch 11, Loss: 4.401142597198486, Final Batch Loss: 2.1981611251831055
Epoch 12, Loss: 4.400351285934448, Final Batch Loss: 2.2069504261016846
Epoch 13, Loss: 4.388543367385864, Final Batch Loss: 2.1843740940093994
Epoch 14, Loss: 4.3819968700408936, Final Batch Loss: 2.1880555152893066
Epoch 

Epoch 118, Loss: 1.8835492730140686, Final Batch Loss: 0.9459570646286011
Epoch 119, Loss: 1.8992271423339844, Final Batch Loss: 0.9814133644104004
Epoch 120, Loss: 1.9106677770614624, Final Batch Loss: 0.9000521898269653
Epoch 121, Loss: 1.864456295967102, Final Batch Loss: 0.9402604103088379
Epoch 122, Loss: 1.9069982171058655, Final Batch Loss: 0.9170746803283691
Epoch 123, Loss: 1.8609440922737122, Final Batch Loss: 0.9239597916603088
Epoch 124, Loss: 1.8266807198524475, Final Batch Loss: 0.9040545225143433
Epoch 125, Loss: 1.8396018147468567, Final Batch Loss: 0.9317435026168823
Epoch 126, Loss: 1.798188865184784, Final Batch Loss: 0.8641582727432251
Epoch 127, Loss: 1.8265233635902405, Final Batch Loss: 0.9104364514350891
Epoch 128, Loss: 1.7593640685081482, Final Batch Loss: 0.8292063474655151
Epoch 129, Loss: 1.7262585163116455, Final Batch Loss: 0.8276712894439697
Epoch 130, Loss: 1.7611960768699646, Final Batch Loss: 0.861130952835083
Epoch 131, Loss: 1.8241919875144958, Fina

Epoch 230, Loss: 1.405604064464569, Final Batch Loss: 0.7253932952880859
Epoch 231, Loss: 1.3772156238555908, Final Batch Loss: 0.6603706479072571
Epoch 232, Loss: 1.4119746088981628, Final Batch Loss: 0.7103503346443176
Epoch 233, Loss: 1.3666326999664307, Final Batch Loss: 0.6779454946517944
Epoch 234, Loss: 1.3745115399360657, Final Batch Loss: 0.6249181032180786
Epoch 235, Loss: 1.3957756161689758, Final Batch Loss: 0.7481343746185303
Epoch 236, Loss: 1.4064461588859558, Final Batch Loss: 0.7402979731559753
Epoch 237, Loss: 1.4284409880638123, Final Batch Loss: 0.7040667533874512
Epoch 238, Loss: 1.415938138961792, Final Batch Loss: 0.7446808218955994
Epoch 239, Loss: 1.363386869430542, Final Batch Loss: 0.6713137626647949
Epoch 240, Loss: 1.4204379320144653, Final Batch Loss: 0.6914213299751282
Epoch 241, Loss: 1.416352927684784, Final Batch Loss: 0.7498769760131836
Epoch 242, Loss: 1.4178376197814941, Final Batch Loss: 0.7733689546585083
Epoch 243, Loss: 1.3667928576469421, Final

Epoch 343, Loss: 1.1426265835762024, Final Batch Loss: 0.5714884400367737
Epoch 344, Loss: 1.132542073726654, Final Batch Loss: 0.5799058675765991
Epoch 345, Loss: 1.1451201438903809, Final Batch Loss: 0.6200394630432129
Epoch 346, Loss: 1.1506258249282837, Final Batch Loss: 0.5716219544410706
Epoch 347, Loss: 1.1486912965774536, Final Batch Loss: 0.5651829242706299
Epoch 348, Loss: 1.2254668474197388, Final Batch Loss: 0.6368802785873413
Epoch 349, Loss: 1.185440182685852, Final Batch Loss: 0.540125846862793
Epoch 350, Loss: 1.1319569945335388, Final Batch Loss: 0.5789538025856018
Epoch 351, Loss: 1.166386365890503, Final Batch Loss: 0.6014363765716553
Epoch 352, Loss: 1.1302145719528198, Final Batch Loss: 0.5474064946174622
Epoch 353, Loss: 1.1184534430503845, Final Batch Loss: 0.4858499765396118
Epoch 354, Loss: 1.0803090333938599, Final Batch Loss: 0.5391020774841309
Epoch 355, Loss: 1.157444179058075, Final Batch Loss: 0.5571220517158508
Epoch 356, Loss: 1.1863152980804443, Final 

Epoch 459, Loss: 1.0346530973911285, Final Batch Loss: 0.4808882176876068
Epoch 460, Loss: 1.1069361567497253, Final Batch Loss: 0.5701172351837158
Epoch 461, Loss: 1.021798551082611, Final Batch Loss: 0.5048655271530151
Epoch 462, Loss: 1.0283113718032837, Final Batch Loss: 0.5346570014953613
Epoch 463, Loss: 1.1227824687957764, Final Batch Loss: 0.5576217770576477
Epoch 464, Loss: 1.0181264579296112, Final Batch Loss: 0.46232661604881287
Epoch 465, Loss: 0.9888880252838135, Final Batch Loss: 0.44032835960388184
Epoch 466, Loss: 1.065086841583252, Final Batch Loss: 0.548753023147583
Epoch 467, Loss: 1.0036338865756989, Final Batch Loss: 0.48790374398231506
Epoch 468, Loss: 1.0241032242774963, Final Batch Loss: 0.4475485682487488
Epoch 469, Loss: 1.05765038728714, Final Batch Loss: 0.5312442183494568
Epoch 470, Loss: 1.0422505140304565, Final Batch Loss: 0.563431978225708
Epoch 471, Loss: 1.0117138028144836, Final Batch Loss: 0.5194341540336609
Epoch 472, Loss: 1.0044430792331696, Fina

...

Epoch 4518, Loss: 0.1668444164097309, Final Batch Loss: 0.1100020632147789
Epoch 4519, Loss: 0.2059701830148697, Final Batch Loss: 0.08603398501873016
Epoch 4520, Loss: 0.23608415573835373, Final Batch Loss: 0.09993945807218552
Epoch 4521, Loss: 0.23288236558437347, Final Batch Loss: 0.09875084459781647
Epoch 4522, Loss: 0.17573243379592896, Final Batch Loss: 0.09130340814590454
Epoch 4523, Loss: 0.18571529537439346, Final Batch Loss: 0.08968599140644073
Epoch 4524, Loss: 0.19986506551504135, Final Batch Loss: 0.0968710407614708
Epoch 4525, Loss: 0.20462344586849213, Final Batch Loss: 0.10711982101202011
Epoch 4526, Loss: 0.2676045298576355, Final Batch Loss: 0.13236522674560547
Epoch 4527, Loss: 0.17843003571033478, Final Batch Loss: 0.07257962971925735
Epoch 4528, Loss: 0.20055659860372543, Final Batch Loss: 0.10855059325695038
Epoch 4529, Loss: 0.20439647883176804, Final Batch Loss: 0.11513368785381317
Epoch 4530, Loss: 0.20000071078538895, Final Batch Loss: 0.08150676637887955
Epoc

Epoch 4636, Loss: 0.15238391235470772, Final Batch Loss: 0.09009139984846115
Epoch 4637, Loss: 0.20141932368278503, Final Batch Loss: 0.10314254462718964
Epoch 4638, Loss: 0.18422629684209824, Final Batch Loss: 0.09022717922925949
Epoch 4639, Loss: 0.20870241522789001, Final Batch Loss: 0.08962642401456833
Epoch 4640, Loss: 0.19989404827356339, Final Batch Loss: 0.09231524914503098
Epoch 4641, Loss: 0.19651976227760315, Final Batch Loss: 0.112610824406147
Epoch 4642, Loss: 0.18813510239124298, Final Batch Loss: 0.10565947741270065
Epoch 4643, Loss: 0.20834166556596756, Final Batch Loss: 0.0881551131606102
Epoch 4644, Loss: 0.17047658562660217, Final Batch Loss: 0.09241954982280731
Epoch 4645, Loss: 0.2607981264591217, Final Batch Loss: 0.1185552328824997
Epoch 4646, Loss: 0.2322218418121338, Final Batch Loss: 0.11372315138578415
Epoch 4647, Loss: 0.16266833990812302, Final Batch Loss: 0.07645126432180405
Epoch 4648, Loss: 0.15770956873893738, Final Batch Loss: 0.08314591646194458
Epoch

Epoch 4749, Loss: 0.28399017453193665, Final Batch Loss: 0.15336918830871582
Epoch 4750, Loss: 0.23474711179733276, Final Batch Loss: 0.13463762402534485
Epoch 4751, Loss: 0.2307855263352394, Final Batch Loss: 0.1300041228532791
Epoch 4752, Loss: 0.26670345664024353, Final Batch Loss: 0.17347675561904907
Epoch 4753, Loss: 0.26941056549549103, Final Batch Loss: 0.12826482951641083
Epoch 4754, Loss: 0.22752974927425385, Final Batch Loss: 0.12028849869966507
Epoch 4755, Loss: 0.20894590020179749, Final Batch Loss: 0.09654133766889572
Epoch 4756, Loss: 0.20990563929080963, Final Batch Loss: 0.09953456372022629
Epoch 4757, Loss: 0.19756988435983658, Final Batch Loss: 0.09372326731681824
Epoch 4758, Loss: 0.1957097202539444, Final Batch Loss: 0.11456121504306793
Epoch 4759, Loss: 0.20837201923131943, Final Batch Loss: 0.11520858108997345
Epoch 4760, Loss: 0.19560855627059937, Final Batch Loss: 0.11533891409635544
Epoch 4761, Loss: 0.15203744918107986, Final Batch Loss: 0.07326235622167587
Ep

Epoch 4859, Loss: 0.26455050706863403, Final Batch Loss: 0.1138475239276886
Epoch 4860, Loss: 0.2599454000592232, Final Batch Loss: 0.12197015434503555
Epoch 4861, Loss: 0.19420651346445084, Final Batch Loss: 0.08276506513357162
Epoch 4862, Loss: 0.15331781655550003, Final Batch Loss: 0.06389104574918747
Epoch 4863, Loss: 0.177162803709507, Final Batch Loss: 0.08785537630319595
Epoch 4864, Loss: 0.22552762925624847, Final Batch Loss: 0.13588951528072357
Epoch 4865, Loss: 0.1968330442905426, Final Batch Loss: 0.08307918906211853
Epoch 4866, Loss: 0.19885173439979553, Final Batch Loss: 0.0743386521935463
Epoch 4867, Loss: 0.33716531842947006, Final Batch Loss: 0.12305449694395065
Epoch 4868, Loss: 0.23019421845674515, Final Batch Loss: 0.12749667465686798
Epoch 4869, Loss: 0.2020956501364708, Final Batch Loss: 0.11966279149055481
Epoch 4870, Loss: 0.17093275487422943, Final Batch Loss: 0.08728935569524765
Epoch 4871, Loss: 0.20697277784347534, Final Batch Loss: 0.11963632702827454
Epoch 

Epoch 4980, Loss: 0.2877914607524872, Final Batch Loss: 0.10279883444309235
Epoch 4981, Loss: 0.20472519099712372, Final Batch Loss: 0.10892688482999802
Epoch 4982, Loss: 0.1781422197818756, Final Batch Loss: 0.1073606088757515
Epoch 4983, Loss: 0.21320762485265732, Final Batch Loss: 0.11308258026838303
Epoch 4984, Loss: 0.23650283366441727, Final Batch Loss: 0.12185713648796082
Epoch 4985, Loss: 0.1839320883154869, Final Batch Loss: 0.09868413954973221
Epoch 4986, Loss: 0.15097910165786743, Final Batch Loss: 0.07208126038312912
Epoch 4987, Loss: 0.25510965287685394, Final Batch Loss: 0.12887461483478546
Epoch 4988, Loss: 0.2018863782286644, Final Batch Loss: 0.10436046123504639
Epoch 4989, Loss: 0.18235743790864944, Final Batch Loss: 0.10025032609701157
Epoch 4990, Loss: 0.21616791188716888, Final Batch Loss: 0.10076625645160675
Epoch 4991, Loss: 0.1842346414923668, Final Batch Loss: 0.07780791819095612
Epoch 4992, Loss: 0.17827627062797546, Final Batch Loss: 0.07654419541358948
Epoch

In [9]:
softmax = nn.Softmax(dim = 1)
model.train()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[15  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0]
 [ 0  0  3  0  0  8  0  0  1]
 [ 0  0  0  0  0  0  7  0  0]
 [ 0  0  0  0  0  0  0 11  0]
 [ 0  0  1  0  0  1  0  0 15]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        15
           1    1.00000   1.00000   1.00000         6
           2    0.71429   1.00000   0.83333        10
           3    1.00000   1.00000   1.00000        16
           4    1.00000   1.00000   1.00000        16
           5    0.88889   0.66667   0.76190        12
           6    1.00000   1.00000   1.00000         7
           7    1.00000   1.00000   1.00000        11
           8    0.93750   0.88235   0.90909        17

    accuracy                        0.94545       110
   macro avg    0.94896   0.94989   0.94493       110
weighted avg    0.95225   0.94545   0.94482       110



# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_Group_5_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [12]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 3)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)

fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [13]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[13  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0]
 [ 0  0  0 15  1  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  2]
 [ 0  0  0  0  1  0 11  0  0]
 [ 0  0  0  0  0  0  0 13  0]
 [ 0  0  0  0  0  0  0  0 10]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        13
           1    1.00000   1.00000   1.00000        19
           2    1.00000   1.00000   1.00000         8
           3    1.00000   0.93750   0.96774        16
           4    0.77778   1.00000   0.87500         7
           5    1.00000   0.83333   0.90909        12
           6    1.00000   0.91667   0.95652        12
           7    1.00000   1.00000   1.00000        13
           8    0.83333   1.00000   0.90909        10

    accuracy                        0.96364       110
   macro avg    0.95679   0.96528   0.95749       110
weighted avg    0.97071   0.96364   0.96443       110

